In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp datasets
!pip install -q -U keras

import os

# Set the backbend before importing Keras
os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

import keras_nlp
import keras

# Run at half precision.
#keras.config.set_floatx("bfloat16")

# Training Configurations
token_limit = 256
num_data_limit = 100
lora_name = "cakeboss"
lora_rank = 4
lr_value = 1e-4
train_epoch = 20
model_id = "gemma2_instruct_2b_en"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import keras
import keras_nlp

import time

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_id)
gemma_lm.summary()

tick_start = 0

def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"TOTAL TIME ELAPSED: {time.time() - tick_start:.2f}s")

def text_gen(prompt):
    tick()
    input = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    output = gemma_lm.generate(input, max_length=token_limit)
    print("\nGemma output:")
    print(output)
    tock()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

# Running Intereference with the three languages before fine tuning

In [3]:
text_gen("వెళ్ళిపోతూ మళ్లీ వస్తానని అన్నాడు. కానీ, అతను తిరిగి రాలేదు. అతని గురించి ఏం అనిపిస్తోంది?")


Gemma output:
<start_of_turn>user
వెళ్ళిపోతూ మళ్లీ వస్తానని అన్నాడు. కానీ, అతను తిరిగి రాలేదు. అతని గురించి ఏం అనిపిస్తోంది?<end_of_turn>
<start_of_turn>model
ఈ పాటలో ఉన్న పాటలో అనేక అంశాలు ఉన్నాయి. అవి కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్నిటిని కొన్ని
TOTAL TIME ELAPSED: 19.54s


In [4]:
text_gen("उसने कहा था कि वो लौटकर आएगा, लेकिन वो वापस नहीं आया। उसके बारे में आपको क्या लगता है?")


Gemma output:
<start_of_turn>user
उसने कहा था कि वो लौटकर आएगा, लेकिन वो वापस नहीं आया। उसके बारे में आपको क्या लगता है?<end_of_turn>
<start_of_turn>model
This is a very open-ended question, and there are many possible interpretations. Here are a few thoughts, depending on the context:

**Possible Interpretations:**

* **He was unreliable:** This is the most straightforward interpretation. He made a promise, but didn't follow through. This could be due to a variety of reasons, from laziness to a lack of commitment.
* **He had a genuine reason for not returning:** Perhaps he faced an unexpected obstacle, a personal crisis, or a situation that made it impossible for him to fulfill his promise. 
* **He was dishonest:** He may have made the promise to manipulate you or gain something from you. 
* **He is no longer interested:** He may have lost interest in the relationship or the situation, and decided to move on.
* **He is in danger:**  This is a more serious interpretation. He may have 

In [5]:
text_gen("उहाँले फर्किन्छु भन्नु भयो, तर उहाँ फर्किनु भएन। तपाईंलाई उहाँबारे के लाग्छ?")


Gemma output:
<start_of_turn>user
उहाँले फर्किन्छु भन्नु भयो, तर उहाँ फर्किनु भएन। तपाईंलाई उहाँबारे के लाग्छ?<end_of_turn>
<start_of_turn>model
This is a classic riddle!  Here's how to break it down:

* **The Setup:** The riddle sets up a situation where someone says they want to leave, but doesn't actually leave. 
* **The Question:** The question asks for the speaker's opinion on the situation.

**The Answer:**  The answer is likely that the person is **stuck**. 

Here's why:

* **Figurative Language:** The riddle uses figurative language to create a sense of confusion.  "Firkna" is a common Nepali word for "to leave" but it's also used to describe a situation where someone is trapped or unable to move.
* **Intrigue:** The riddle is designed to make you think about literal leaving, but the focus is on the speaker's inability to do so.


Let me know if you'd like to try another riddle! 
<end_of_turn>
TOTAL TIME ELAPSED: 6.26s
